In [8]:
import os
import logging
import csv
from datetime import datetime, timedelta, timezone
from binance_sdk_c2c.c2c import C2C, ConfigurationRestAPI, C2C_REST_API_PROD_URL

# Configure logging
logging.basicConfig(level=logging.INFO)

# Create configuration for the REST API
configuration_rest_api = ConfigurationRestAPI(
    api_key=os.getenv("API_KEY", ""),
    api_secret=os.getenv("API_SECRET", ""),
    base_path=os.getenv("BASE_PATH", C2C_REST_API_PROD_URL),
)

# Initialize C2C client
client = C2C(config_rest_api=configuration_rest_api)

def get_c2_c_trade_history_for_day():
    try:
        # Define time range for 23/09/2025 (UTC+7)
        tz_vietnam = timezone(timedelta(hours=7))
        start_of_day = datetime(2025, 9, 23, 0, 0, 0, tzinfo=tz_vietnam)
        end_of_day = start_of_day + timedelta(days=1) - timedelta(milliseconds=1)
        
        start_time = int(start_of_day.timestamp() * 1000)  # Convert to milliseconds
        end_time = int(end_of_day.timestamp() * 1000)      # Convert to milliseconds

        all_trades = []
        page = 1
        rows = 200  # Maximum records per page

        while True:
            response = client.rest_api.get_c2_c_trade_history(
                start_time=start_time,
                end_time=end_time,
                page=page,
                rows=rows,  # Set to maximum allowed
                recv_window=60000
            )

            rate_limits = response.rate_limits
            logging.info(f"Page {page} rate limits: {rate_limits}")

            # Extract the inner data list from the response
            response_data = response.data()
            if response_data.code != '000000' or not response_data.data:
                logging.info(f"Stopping at page {page}: No more data or error (code: {response_data.code})")
                break

            data = response_data.data
            all_trades.extend(data)
            logging.info(f"Page {page} retrieved {len(data)} records")

            # If fewer than rows records, we've reached the end
            if len(data) < rows:
                break

            page += 1

        # Write data to CSV file
        if all_trades:
            output_file = "c2c_trade_history_20250923.csv"
            headers = [
                'order_number', 'adv_no', 'trade_type', 'asset', 'fiat', 
                'fiat_symbol', 'amount', 'total_price', 'unit_price', 
                'order_status', 'create_time', 'commission', 
                'counter_part_nick_name', 'advertisement_role'
            ]
            
            with open(output_file, 'w', newline='', encoding='utf-8') as csvfile:
                writer = csv.DictWriter(csvfile, fieldnames=headers)
                writer.writeheader()
                
                for item in all_trades:
                    writer.writerow({
                        'order_number': item.order_number,
                        'adv_no': item.adv_no,
                        'trade_type': item.trade_type,
                        'asset': item.asset,
                        'fiat': item.fiat,
                        'fiat_symbol': item.fiat_symbol,
                        'amount': item.amount,
                        'total_price': item.total_price,
                        'unit_price': item.unit_price,
                        'order_status': item.order_status,
                        'create_time': item.create_time,  # Keep as timestamp; convert in Excel if needed
                        'commission': item.commission,
                        'counter_part_nick_name': item.counter_part_nick_name,
                        'advertisement_role': item.advertisement_role
                    })
                    
            logging.info(f"Trade history written to {output_file} with {len(all_trades)} records")
        else:
            logging.warning("No data to write to CSV")

    except Exception as e:
        logging.error(f"Error fetching trades for 23/09/2025: {str(e)}")

if __name__ == "__main__":
    get_c2_c_trade_history_for_day()

ERROR:root:Error fetching trades for 23/09/2025: C2CRestAPI.get_c2_c_trade_history() got an unexpected keyword argument 'rows'


In [9]:
# Cell 1: Fetch all trade history for September 23, 2025 and save to CSV
import os
import logging
import csv
from datetime import datetime, timedelta, timezone
from binance_sdk_c2c.c2c import C2C, ConfigurationRestAPI, C2C_REST_API_PROD_URL

# Configure logging
logging.basicConfig(level=logging.INFO)

# Create configuration for the REST API
configuration_rest_api = ConfigurationRestAPI(
    api_key=os.getenv("API_KEY", ""),
    api_secret=os.getenv("API_SECRET", ""),
    base_path=os.getenv("BASE_PATH", C2C_REST_API_PROD_URL),
)

# Initialize C2C client
client = C2C(config_rest_api=configuration_rest_api)

def get_c2_c_trade_history_for_day():
    try:
        # Define time range for 23/09/2025 (UTC+7)
        tz_vietnam = timezone(timedelta(hours=7))
        start_of_day = datetime(2025, 9, 23, 0, 0, 0, tzinfo=tz_vietnam)
        end_of_day = start_of_day + timedelta(days=1) - timedelta(milliseconds=1)
        
        start_time = int(start_of_day.timestamp() * 1000)  # Convert to milliseconds
        end_time = int(end_of_day.timestamp() * 1000)      # Convert to milliseconds

        all_trades = []
        page = 1
        max_records = 50  # Assume 50 records per page based on observed output

        while True:
            response = client.rest_api.get_c2_c_trade_history(
                start_time=start_time,
                end_time=end_time,
                page=page,
                recv_window=60000
            )

            rate_limits = response.rate_limits
            logging.info(f"Page {page} rate limits: {rate_limits}")

            # Extract the inner data list from the response
            response_data = response.data()
            if response_data.code != '000000' or not response_data.data:
                logging.info(f"Stopping at page {page}: No more data or error (code: {response_data.code})")
                break

            data = response_data.data
            all_trades.extend(data)
            logging.info(f"Page {page} retrieved {len(data)} records")

            # If fewer than max_records, we've reached the end
            if len(data) < max_records:
                break

            page += 1

        # Write data to CSV file
        if all_trades:
            output_file = "c2c_trade_history_20250923.csv"
            headers = [
                'order_number', 'adv_no', 'trade_type', 'asset', 'fiat', 
                'fiat_symbol', 'amount', 'total_price', 'unit_price', 
                'order_status', 'create_time', 'commission', 
                'counter_part_nick_name', 'advertisement_role'
            ]
            
            with open(output_file, 'w', newline='', encoding='utf-8') as csvfile:
                writer = csv.DictWriter(csvfile, fieldnames=headers)
                writer.writeheader()
                
                for item in all_trades:
                    writer.writerow({
                        'order_number': item.order_number,
                        'adv_no': item.adv_no,
                        'trade_type': item.trade_type,
                        'asset': item.asset,
                        'fiat': item.fiat,
                        'fiat_symbol': item.fiat_symbol,
                        'amount': item.amount,
                        'total_price': item.total_price,
                        'unit_price': item.unit_price,
                        'order_status': item.order_status,
                        'create_time': item.create_time,  # Keep as timestamp; convert in Excel if needed
                        'commission': item.commission,
                        'counter_part_nick_name': item.counter_part_nick_name,
                        'advertisement_role': item.advertisement_role
                    })
                    
            logging.info(f"Trade history written to {output_file} with {len(all_trades)} records")
        else:
            logging.warning("No data to write to CSV")

    except Exception as e:
        logging.error(f"Error fetching trades for 23/09/2025: {str(e)}")

if __name__ == "__main__":
    get_c2_c_trade_history_for_day()

INFO:root:Page 1 rate limits: []
INFO:root:Page 1 retrieved 50 records
INFO:root:Page 2 rate limits: []
INFO:root:Page 2 retrieved 50 records
INFO:root:Page 3 rate limits: []
INFO:root:Page 3 retrieved 50 records
INFO:root:Page 4 rate limits: []
INFO:root:Page 4 retrieved 50 records
INFO:root:Page 5 rate limits: []
INFO:root:Page 5 retrieved 50 records
INFO:root:Page 6 rate limits: []
INFO:root:Page 6 retrieved 50 records
INFO:root:Page 7 rate limits: []
INFO:root:Page 7 retrieved 50 records
INFO:root:Page 8 rate limits: []
INFO:root:Page 8 retrieved 50 records
INFO:root:Page 9 rate limits: []
INFO:root:Page 9 retrieved 50 records
INFO:root:Page 10 rate limits: []
INFO:root:Page 10 retrieved 50 records
INFO:root:Page 11 rate limits: []
INFO:root:Page 11 retrieved 50 records
INFO:root:Page 12 rate limits: []
INFO:root:Page 12 retrieved 50 records
INFO:root:Page 13 rate limits: []
INFO:root:Page 13 retrieved 50 records
INFO:root:Page 14 rate limits: []
INFO:root:Page 14 retrieved 50 reco

In [ ]:
# Cell 1: Fetch trade history for today (from 00:00 to current time) and save to CSV
import os
import logging
import csv
from datetime import datetime, timedelta, timezone
from binance_sdk_c2c.c2c import C2C, ConfigurationRestAPI, C2C_REST_API_PROD_URL

# Configure logging
logging.basicConfig(level=logging.INFO)

# Create configuration for the REST API
configuration_rest_api = ConfigurationRestAPI(
    api_key=os.getenv("API_KEY", ""),
    api_secret=os.getenv("API_SECRET", ""),
    base_path=os.getenv("BASE_PATH", C2C_REST_API_PROD_URL),
)

# Initialize C2C client
client = C2C(config_rest_api=configuration_rest_api)

def get_c2_c_trade_history_for_today():
    try:
        # Define time range for today from 00:00 to current time (UTC+7)
        tz_vietnam = timezone(timedelta(hours=7))
        now = datetime.now(tz_vietnam)  # Current time in UTC+7
        start_of_day = now.replace(hour=0, minute=0, second=0, microsecond=0)  # 00:00 today
        
        start_time = int(start_of_day.timestamp() * 1000)  # Convert to milliseconds
        end_time = int(now.timestamp() * 1000)            # Convert to milliseconds

        all_trades = []
        page = 1
        max_records = 50  # Assume 50 records per page based on previous observation

        while True:
            response = client.rest_api.get_c2_c_trade_history(
                start_time=start_time,
                end_time=end_time,
                page=page,
                recv_window=60000
            )

            rate_limits = response.rate_limits
            logging.info(f"Page {page} rate limits: {rate_limits}")

            # Extract the inner data list from the response
            response_data = response.data()
            if response_data.code != '000000' or not response_data.data:
                logging.info(f"Stopping at page {page}: No more data or error (code: {response_data.code})")
                break

            data = response_data.data
            all_trades.extend(data)
            logging.info(f"Page {page} retrieved {len(data)} records")

            # If fewer than max_records, we've reached the end
            if len(data) < max_records:
                break

            page += 1

        # Write data to CSV file
        if all_trades:
            output_file = f"c2c_trade_history_{now.strftime('%Y%m%d')}.csv"
            headers = [
                'order_number', 'adv_no', 'trade_type', 'asset', 'fiat', 
                'fiat_symbol', 'amount', 'total_price', 'unit_price', 
                'order_status', 'create_time', 'commission', 
                'counter_part_nick_name', 'advertisement_role'
            ]
            
            with open(output_file, 'w', newline='', encoding='utf-8') as csvfile:
                writer = csv.DictWriter(csvfile, fieldnames=headers)
                writer.writeheader()
                
                for item in all_trades:
                    writer.writerow({
                        'order_number': item.order_number,
                        'adv_no': item.adv_no,
                        'trade_type': item.trade_type,
                        'asset': item.asset,
                        'fiat': item.fiat,
                        'fiat_symbol': item.fiat_symbol,
                        'amount': item.amount,
                        'total_price': item.total_price,
                        'unit_price': item.unit_price,
                        'order_status': item.order_status,
                        'create_time': item.create_time,  # Keep as timestamp; convert in Excel if needed
                        'commission': item.commission,
                        'counter_part_nick_name': item.counter_part_nick_name,
                        'advertisement_role': item.advertisement_role
                    })
                    
            logging.info(f"Trade history written to {output_file} with {len(all_trades)} records")
        else:
            logging.warning("No data to write to CSV")

    except Exception as e:
        logging.error(f"Error fetching trades for today ({now.strftime('%Y-%m-%d')}): {str(e)}")

if __name__ == "__main__":
    get_c2_c_trade_history_for_today()

In [22]:
tz_vietnam = timezone(timedelta(hours=7))
tz_vietnam

datetime.timezone(datetime.timedelta(seconds=25200))

In [23]:
now = datetime.now(tz_vietnam)
now

datetime.datetime(2025, 9, 23, 20, 5, 13, 881495, tzinfo=datetime.timezone(datetime.timedelta(seconds=25200)))

In [25]:
start_of_day = now.replace(hour=0, minute=0, second=0, microsecond=0)
start_of_day


datetime.datetime(2025, 9, 23, 0, 0, tzinfo=datetime.timezone(datetime.timedelta(seconds=25200)))

In [32]:
# Cell 1: Fetch trade history for yesterday (22/09/2025) and save to CSV
import os
import logging
import csv
from datetime import datetime, timedelta, timezone
from binance_sdk_c2c.c2c import C2C, ConfigurationRestAPI, C2C_REST_API_PROD_URL

# Configure logging
logging.basicConfig(level=logging.INFO)

# Create configuration for the REST API
configuration_rest_api = ConfigurationRestAPI(
    api_key=os.getenv("API_KEY", ""),
    api_secret=os.getenv("API_SECRET", ""),
    base_path=os.getenv("BASE_PATH", C2C_REST_API_PROD_URL),
)

# Initialize C2C client
client = C2C(config_rest_api=configuration_rest_api)

def get_c2_c_trade_history_for_yesterday():
    try:
        # Define time range for yesterday (22/09/2025, UTC+7)
        tz_vietnam = timezone(timedelta(hours=7))
        today = datetime(2025, 9, 23, tzinfo=tz_vietnam)  # Hard-code to match current date
        # start_of_yesterday = (today - timedelta(days=1)).replace(hour=0, minute=0, second=0, microsecond=0)
        # end_of_yesterday = start_of_yesterday + timedelta(days=1) - timedelta(milliseconds=1)
        end_of_today = today.replace(hour=23, minute=59, second=59, microsecond=999000)
        start_time = int(today.timestamp() * 1000)  # Convert to milliseconds
        end_time = int(end_of_today.timestamp() * 1000)      # Convert to milliseconds

        logging.info(f"Fetching trades from {today.strftime('%Y-%m-%d %H:%M:%S %Z')} "
                     f"to {end_of_today.strftime('%Y-%m-%d %H:%M:%S %Z')}")

        all_trades = []
        page = 1
        max_records = 50  # Assume 50 records per page based on previous observation

        while True:
            response = client.rest_api.get_c2_c_trade_history(
                start_time=start_time,
                end_time=end_time,
                page=page,
                recv_window=60000
            )

            rate_limits = response.rate_limits
            logging.info(f"Page {page} rate limits: {rate_limits}")

            # Extract the inner data list from the response
            response_data = response.data()
            if response_data.code != '000000' or not response_data.data:
                logging.info(f"Stopping at page {page}: No more data or error (code: {response_data.code})")
                break

            data = response_data.data
            # Verify that trades are within the requested time range
            filtered_data = [
                trade for trade in data
                if start_time <= trade.create_time <= end_time
            ]
            all_trades.extend(filtered_data)
            logging.info(f"Page {page} retrieved {len(data)} records, {len(filtered_data)} within time range")

            # If fewer than max_records, we've reached the end
            if len(data) < max_records:
                break

            page += 1

        # Write data to CSV file
        if all_trades:
            output_file = "today_data.csv"
            headers = [
                'order_number', 'adv_no', 'trade_type', 'asset', 'fiat', 
                'fiat_symbol', 'amount', 'total_price', 'unit_price', 
                'order_status', 'create_time', 'commission', 
                'counter_part_nick_name', 'advertisement_role'
            ]
            
            with open(output_file, 'w', newline='', encoding='utf-8') as csvfile:
                writer = csv.DictWriter(csvfile, fieldnames=headers)
                writer.writeheader()
                
                for item in all_trades:
                    writer.writerow({
                        'order_number': item.order_number,
                        'adv_no': item.adv_no,
                        'trade_type': item.trade_type,
                        'asset': item.asset,
                        'fiat': item.fiat,
                        'fiat_symbol': item.fiat_symbol,
                        'amount': item.amount,
                        'total_price': item.total_price,
                        'unit_price': item.unit_price,
                        'order_status': item.order_status,
                        'create_time': item.create_time,  # Keep as timestamp; convert in Excel if needed
                        'commission': item.commission,
                        'counter_part_nick_name': item.counter_part_nick_name,
                        'advertisement_role': item.advertisement_role
                    })
                    
            logging.info(f"Trade history written to {output_file} with {len(all_trades)} records")
        else:
            logging.warning("No data to write to CSV for 22/09/2025")

    except Exception as e:
        logging.error(f"Error fetching trades for 22/09/2025: {str(e)}")

if __name__ == "__main__":
    get_c2_c_trade_history_for_yesterday()

INFO:root:Fetching trades from 2025-09-23 00:00:00 UTC+07:00 to 2025-09-23 23:59:59 UTC+07:00
INFO:root:Page 1 rate limits: []
INFO:root:Page 1 retrieved 50 records, 50 within time range
INFO:root:Page 2 rate limits: []
INFO:root:Page 2 retrieved 50 records, 50 within time range
INFO:root:Page 3 rate limits: []
INFO:root:Page 3 retrieved 50 records, 50 within time range
INFO:root:Page 4 rate limits: []
INFO:root:Page 4 retrieved 50 records, 50 within time range
INFO:root:Page 5 rate limits: []
INFO:root:Page 5 retrieved 50 records, 50 within time range
INFO:root:Page 6 rate limits: []
INFO:root:Page 6 retrieved 50 records, 50 within time range
INFO:root:Page 7 rate limits: []
INFO:root:Page 7 retrieved 50 records, 50 within time range
INFO:root:Page 8 rate limits: []
INFO:root:Page 8 retrieved 50 records, 50 within time range
INFO:root:Page 9 rate limits: []
INFO:root:Page 9 retrieved 50 records, 50 within time range
INFO:root:Page 10 rate limits: []
INFO:root:Page 10 retrieved 50 reco

In [8]:
import os
import logging
import csv
from datetime import datetime, timedelta, timezone


from binance_sdk_c2c.c2c import C2C, ConfigurationRestAPI, C2C_REST_API_PROD_URL

In [2]:
pwd

'd:\\Working\\binance-merchant-trading-flow\\notebook'

In [3]:
os.chdir("../")

In [4]:
from turtle import home
from typing import List, Optional
from binance_sdk_c2c.rest_api.models import GetC2CTradeHistoryResponse, GetC2CTradeHistoryResponseDataInner
from src.utils.utils import *

In [15]:

class C2CExtended(C2C):
    """Extended C2C API with specific time-based trade history retrieval methods for Vietnam timezone (UTC+7)."""

    def __init__(self, config_rest_api: ConfigurationRestAPI = None) -> None:
        super().__init__(config_rest_api)
        self.max_records = 50  # Maximum records per request as per observed API limit
        self.tz_vietnam = timezone(timedelta(hours=7))  # UTC+7 For Vietnam timezone

    def _fetch_data(
        self,
        start_time: Optional[int] = None,
        end_time: Optional[int] = None,
    ) -> List[GetC2CTradeHistoryResponseDataInner]:
        """
        Fetch all trade history records with pagination.
        
        Args:
            start_time (Optional[int]): Start timestamp in milliseconds
            end_time (Optional[int]): End timestamp in milliseconds
            
        Returns:
            List[GetC2CTradeHistoryResponseDataInner]: List of all trade records
        """
        fetch_data = []
        page = 1 

        while True:
            response = self.rest_api.get_c2_c_trade_history(
                start_time=start_time,
                end_time=end_time,
                page=page,
                recv_window=60000
            )

            rate_limits = response.rate_limits
            logging.info(f"Page {page} rate limits: {rate_limits}")

            # Extract the inner data list from the response
            response_data = response.data()
            if response_data.code != '000000' or not response_data.data:
                logging.info(f"Stopping at page {page}: No more data or error (code: {response_data.code})")
                break

            data = response_data.data
            # Verify that trades are within the requested time range
            filtered_data = [
                trade for trade in data
                if start_time <= trade.create_time <= end_time
            ]
            fetch_data.extend(filtered_data)
            logging.info(f"Page {page} retrieved {len(data)} records, {len(filtered_data)} within time range")

            # If fewer than max_records, we've reached the end
            if len(data) < self.max_records:
                break

            page += 1
        
        return fetch_data

    def get_latest(self) -> List[GetC2CTradeHistoryResponseDataInner]:
        """
        Get trade history from 00:00 of current day to now in Vietnam timezone (UTC+7).
        Includes both BUY and SELL trades.
        
        Returns:
            List[GetC2CTradeHistoryResponseDataInner]: List of trade records
        """
        now = datetime.now(self.tz_vietnam)
        start_of_today = now.replace(hour=0, minute=0, second=0, microsecond=0)
        end_of_today = now.replace(hour=23, minute=59, second=59, microsecond=999000)

        start_time = get_timestamp(start_of_today)
        end_time = get_timestamp(end_of_today)

        return self._fetch_data(start_time, end_time)
    

    def get_latest_by_week(self) -> List[GetC2CTradeHistoryResponseDataInner]:
        """
        Get trade history from start of current week (Monday) to now in Vietnam timezone (UTC+7).
        Includes both BUY and SELL trades.
        
        Returns:
            List[GetC2CTradeHistoryResponseDataInner]: List of trade records
        """

        now = datetime.now(self.tz_vietnam)

        # Calculate days since Monday (0 = Mon, 1 = Tues,...)
        days_since_monday = now.weekday()
        start_of_week = (now - timedelta(days=days_since_monday)).replace(hour=0, minute=0, second=0, microsecond=0)

        start_time = get_timestamp(start_of_week)
        end_time = get_timestamp(now)

        return self._fetch_data(start_time, end_time)
    
    def get_prev_week_data(self) -> List[GetC2CTradeHistoryResponseDataInner]:
        """
        Get trade history for the entire previous week (Monday to Sunday) in Vietnam timezone (UTC+7).
        Includes both BUY and SELL trades.
        
        Returns:
            List[GetC2CTradeHistoryResponseDataInner]: List of trade records
        """
        now = datetime.now(self.tz_vietnam)
        days_since_monday = now.weekday()
        start_of_current_week = (now - timedelta(days=days_since_monday)).replace(hour=0, minute=0, second=0, microsecond=0)
        start_of_prev_week = start_of_current_week - timedelta(days=7)
        end_of_prev_week = start_of_prev_week + timedelta(days=7) - timedelta(milliseconds=1)

        start_time = get_timestamp(start_of_prev_week)
        end_time = get_timestamp(end_of_prev_week)

        return self._fetch_data(start_time, end_time)

    def get_latest_by_month(self) -> List[GetC2CTradeHistoryResponseDataInner]:
        """
        Get trade history from start of current month to now in Vietnam timezone (UTC+7).
        Includes both BUY and SELL trades.
        
        Returns:
            List[GetC2CTradeHistoryResponseDataInner]: List of trade records
        """
        now = datetime.now(self.tz_vietnam)

        start_of_month = now.replace(day=1, hour=0, minute=0, second=0, microsecond=0)

        start_time = get_timestamp(start_of_month)
        end_time = get_timestamp(now)

        return self._fetch_data(start_time, end_time)

    def get_prev_month(self) -> List[GetC2CTradeHistoryResponseDataInner]:
        """
        Get trade history for the entire previous month in Vietnam timezone (UTC+7).
        Includes both BUY and SELL trades.
        
        Returns:
            List[GetC2CTradeHistoryResponseDataInner]: List of trade records
        """

        now = datetime.now(self.tz_vietnam)

        # Get first day of current month
        start_of_current_month = now.replace(day=1, hour=0, minute=0, second=0, microsecond=0)

        # Get first day of previous month
        start_of_prev_month = (start_of_current_month - timedelta(days=1)).replace(day=1)

        # Get last day of previous month
        if start_of_prev_month.month == 12:
            end_of_prev_month = start_of_prev_month.replace(year=start_of_prev_month.year + 1,
            month=1,
            day=1)
        else:
            end_of_prev_month = start_of_prev_month.replace(month=start_of_prev_month.month + 1,
            day=1)
        
        end_of_prev_month -= timedelta(milliseconds=1)

        start_time = get_timestamp(start_of_prev_month)
        end_time = get_timestamp(end_of_prev_month)

        return self._fetch_data(start_time, end_time)

    def get_yesterday(self) -> List[GetC2CTradeHistoryResponseDataInner]:
        """
        Get trade history for the yesterday since today in Vietnam timezone (UTC+7).
        Includes both BUY and SELL trades.
        
        Returns:
            List[GetC2CTradeHistoryResponseDataInner]: List of trade records
        """

        now = datetime.now(self.tz_vietnam)
        
        # Start of yesterday
        start_of_yesterday = (now - timedelta(days=1)).replace(hour=0, 
                                                            minute=0,
                                                            second=0,
                                                            microsecond=0)
        
        # End of yesterday
        end_of_yesterday = start_of_yesterday.replace(hour=23,
                                                    minute=59,
                                                    second=59,
                                                    microsecond=999000)

        start_time = get_timestamp(start_of_yesterday)
        end_time = get_timestamp(end_of_yesterday)

        return self._fetch_data(start_time, end_time)



In [16]:
from binance_common.configuration import ConfigurationRestAPI
# Create configuration for the REST API
configuration_rest_api = ConfigurationRestAPI(
    api_key=os.getenv("API_KEY", ""),
    api_secret=os.getenv("API_SECRET", ""),
    base_path=os.getenv("BASE_PATH", C2C_REST_API_PROD_URL),
)
c2c = C2CExtended(configuration_rest_api)
data = c2c.get_yesterday()

INFO:root:Page 1 rate limits: []
INFO:root:Page 1 retrieved 50 records, 0 within time range
INFO:root:Page 2 rate limits: []
INFO:root:Page 2 retrieved 50 records, 34 within time range
INFO:root:Page 3 rate limits: []
INFO:root:Page 3 retrieved 50 records, 50 within time range
INFO:root:Page 4 rate limits: []
INFO:root:Page 4 retrieved 50 records, 50 within time range
INFO:root:Page 5 rate limits: []
INFO:root:Page 5 retrieved 50 records, 50 within time range
INFO:root:Page 6 rate limits: []
INFO:root:Page 6 retrieved 50 records, 50 within time range
INFO:root:Page 7 rate limits: []
INFO:root:Page 7 retrieved 50 records, 50 within time range
INFO:root:Page 8 rate limits: []
INFO:root:Page 8 retrieved 50 records, 50 within time range
INFO:root:Page 9 rate limits: []
INFO:root:Page 9 retrieved 50 records, 50 within time range
INFO:root:Page 10 rate limits: []
INFO:root:Page 10 retrieved 50 records, 50 within time range
INFO:root:Page 11 rate limits: []
INFO:root:Page 11 retrieved 50 reco

In [17]:
output_file = write_to_csv(data, "2025-09-27")

INFO:root:Trade history written to c2c_trade_history_20250927.csv with 503 records


In [14]:
tz_vietnam = timezone(timedelta(hours=7))
now = datetime.now(tz_vietnam)

In [8]:
days_since_monday = now.weekday()
days_since_monday

1

In [10]:
start_of_week = (now - timedelta(days=days_since_monday)).replace(hour=0, minute=0, second=0, microsecond=0)
start_of_week

datetime.datetime(2025, 9, 22, 0, 0, tzinfo=datetime.timezone(datetime.timedelta(seconds=25200)))